### Importing the required modules

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Building the CNN Network

In [54]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 10, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2)
        # self.gap=torch.nn.AvgPool2d(kernel_size=7, stride=1, padding=0)
        self.bn5=nn.BatchNorm2d(10)
        self.bn6=nn.BatchNorm2d(10)
        self.fc1=  nn.Linear(7*7*10,20)
        self.fc2 = nn.Linear(20, 10)
    def forward(self, x):
        x = self.bn3(F.relu(self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))))
        x=self.bn6(F.relu(self.pool2(self.bn5(F.relu(self.conv4(x))))))
        x = x.view(-1, 7*7*10)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return F.log_softmax(x)

### Checking the total no of parameters

In [55]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
       BatchNorm2d-4           [-1, 32, 28, 28]              64
         MaxPool2d-5           [-1, 32, 14, 14]               0
       BatchNorm2d-6           [-1, 32, 14, 14]              64
            Conv2d-7           [-1, 10, 14, 14]           2,890
       BatchNorm2d-8           [-1, 10, 14, 14]              20
         MaxPool2d-9             [-1, 10, 7, 7]               0
      BatchNorm2d-10             [-1, 10, 7, 7]              20
           Linear-11                   [-1, 20]           9,820
           Linear-12                   [-1, 10]             210
Total params: 17,920
Trainable params: 17,920
Non-trainable params: 0
---------------------------------

### Downloading the data

In [12]:
torch.manual_seed(42)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomApply([transforms.CenterCrop(22), ], p=0.1),
                        transforms.Resize((28, 28)),
                        transforms.RandomRotation((-15., 15.), fill=0),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


### Training the model

In [59]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    accuracy= 100. * correct / len(test_loader.dataset)
    return accuracy

In [62]:
import warnings
warnings.filterwarnings("ignore")

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.85)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1, verbose=True)

for epoch in range(1, 20):
    print(f"epoch======================================>{epoch}")
    train(model, device, train_loader, optimizer, epoch)
    accuracy=test(model, device, test_loader)
    if accuracy>=99.40:
        break
    scheduler.step()

Adjusting learning rate of group 0 to 1.0000e-02.
epoch======================================>1


loss=0.07345708459615707 batch_id=468: 100%|██████████| 469/469 [00:26<00:00, 17.97it/s]



Test set: Average loss: 0.0665, Accuracy: 9791/10000 (97.91%)

Adjusting learning rate of group 0 to 1.0000e-02.
epoch======================================>2


loss=0.09398254007101059 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.27it/s]



Test set: Average loss: 0.0371, Accuracy: 9883/10000 (98.83%)

Adjusting learning rate of group 0 to 1.0000e-02.
epoch======================================>3


...


Test set: Average loss: 0.0181, Accuracy: 9934/10000 (99.34%)

Adjusting learning rate of group 0 to 1.0000e-03.
epoch======================================>18


loss=0.011003772728145123 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.39it/s]



Test set: Average loss: 0.0175, Accuracy: 9935/10000 (99.35%)

Adjusting learning rate of group 0 to 1.0000e-03.
epoch======================================>19


loss=0.017421146854758263 batch_id=468: 100%|██████████| 469/469 [00:25<00:00, 18.39it/s]



Test set: Average loss: 0.0174, Accuracy: 9941/10000 (99.41%)

